In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

In [5]:
df = pd.read_csv("HomeC2.csv")
df['time'] = pd.to_datetime(df['time'])
df = df.set_index('time')
df.head(2)

,House overall,Dishwasher,Home office,Fridge,Wine cellar,Garage door,Barn,Well,Microwave,Living room,...,cloudCover,windBearing,precipIntensity,dewPoint,precipProbability,month,day,weekday,hour,minute
time,,,,,,,,,,,,,,,,,,,,,
2016-01-01 14:00:00,0.932833,0.000033,0.442633,0.12415,0.006983,0.013083,0.03135,0.001017,0.004067,0.001517,...,0.75,282.0,0.0,24.4,0.0,1,1,Friday,14,0
2016-01-01 14:01:00,0.934333,0.000000,0.444067,0.12400,0.006983,0.013117,0.03150,0.001017,0.004067,0.001650,...,0.75,282.0,0.0,24.4,0.0,1,1,Friday,14,1


In [6]:
data = df[['Fridge', 'temperature', 'humidity',
       'visibility', 'apparentTemperature', 'pressure', 'windSpeed',
       'cloudCover', 'windBearing', 'precipIntensity', 'dewPoint',
       'precipProbability']]

In [7]:
scaler = MinMaxScaler(feature_range=(0, 1))
data[data.columns[1:]] = scaler.fit_transform(data[data.columns[1:]])
scaler_target = MinMaxScaler(feature_range=(0, 1))
data[['Fridge']] = scaler_target.fit_transform(data[['Fridge']])

In [8]:
size = int(len(df.loc[:'2016-11-1 00:00']))
data_train = data[:size]
data_test = data[size:]
X_train, X_test = [], []
Y_train, Y_test = [], []
n_past=1
n_future=1
for i in range(n_past, len(data_train)-n_future+1):
    X_train.append(data_train.iloc[i-n_past:i, 0:data.shape[1]])
    Y_train.append(data_train.iloc[i+n_future-1:i+n_future, 0])
for i in range(n_past, len(data_test)-n_future+1):
    X_test.append(data_test.iloc[i-n_past:i, 0:data_test.shape[1]])
    Y_test.append(data_test.iloc[i+n_future-1:i+n_future, 0])
    
X_train, Y_train = np.array(X_train), np.array(Y_train)
X_test, Y_test = np.array(X_test), np.array(Y_test)
print('X_train shape', X_train.shape)
print('X_test shape', X_test.shape)
print('Y_train shape', Y_train.shape)
print('Y_test shape', Y_test.shape)

KeyboardInterrupt: 

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Dropout
from keras.layers import Bidirectional

In [ ]:
model = Sequential()
model.add(LSTM(25, activation='relu', return_sequences = False, input_shape=(X_train.shape[1], X_train.shape[2])))
#model.add(Dropout(0.2))
model.add(Dense(Y_train.shape[1]))
model.compile(optimizer='adam', loss='mse')
model.summary()

In [ ]:
model_fit = model.fit(X_train, Y_train, epochs=10, verbose=0)
Train_pred = model.predict(X_train, verbose=0)
Y_pred = model.predict(X_test, verbose=0)
plt.plot(model_fit.history['loss'], label='Train loss')
plt.legend()
print('Train MSE minimum:', min(model_fit.history['loss']))

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
data[['Fridge']] = scaler_target.inverse_transform(data[['Fridge']])
Y_pred = scaler_target.inverse_transform(Y_pred)
Train_pred = scaler_target.inverse_transform(Train_pred)
plt.figure(figsize=(15,4))
plt.plot(data[['Fridge']][size:-1].values)
plt.plot(Y_pred)
np.sqrt(mean_squared_error(Y_pred[:,0].tolist(), data[['Fridge']][size:-1].values))

In [ ]:
Y_pred_series = pd.Series(Y_pred.flatten().tolist(), index=data['Fridge'][size:-n_past].index)
Train_pred_series = pd.Series(Train_pred.flatten().tolist(), index=data['Fridge'][n_past:size].index)
plt.figure(figsize=(15,4))
plt.plot(data['Fridge'][:-n_past], c='blue', label='data')
plt.plot(Y_pred_series, c='red', label='model test')
plt.plot(Train_pred_series, c='green', label='model train')
plt.legend()
plt.grid(), plt.margins(x=0);
Y_test = data['Fridge'][size:-n_past]

# calcolo errore
print('MSE: %.5f' % (mean_squared_error(Y_pred, Y_test)))
print('RMSE: %.5f' % np.sqrt(mean_squared_error(Y_pred, Y_test)))
MAE = mean_absolute_error(Y_test, Y_pred)
MAPE = np.mean(np.abs(Y_pred[:,0] - Y_test.values)/np.abs(Y_test.values))
print('MAE: %.3f' % MAE)
print('MAPE: %.3f' %MAPE)
print('R^2 score: %.3f' % r2_score(Y_test, Y_pred))

In [ ]:
d1 = plt.figure(figsize=(15,4))
plt.plot(data['Fridge']['2016-11-1'], c='blue', label='data')
plt.plot(Y_pred_series['2016-11-1'], c='red', label='model test')

In [ ]:
df_r = data['Fridge']['2016-11']
df_r = pd.DataFrame({"Fridge_r":df_r})
df_p = Y_pred_series['2016-11']
df_p = pd.DataFrame({"Fridge_p":df_p})
df_t = pd.merge(df_r, df_p, how='inner', on='time')
df_t.to_csv('Device3.csv', index=False)

In [ ]:
df_t